In [ ]:
import sys
sys.path.append("/storage/vbutoi/projects/")
sys.path.append("/storage/vbutoi/libraries/")
from ionpy.analysis import ResultsLoader
import seaborn as sns
import pandas as pd
sns.set_context('talk')

sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage',
))

# Results loader object does everything
rs = ResultsLoader()
root = "/storage/vbutoi/scratch/ESE"

In [ ]:
# path = f"{root}/training/01_03_24_WMH_Ensemble"
path = f"{root}/calibration/01_03_24_WMH_LongerCalibrationSuite"

dfc = rs.load_configs(
    path,
    properties=False,
)
df = rs.load_metrics(dfc)

In [ ]:
df

In [ ]:
# def has_aug(augmentations):
#     return not(augmentations == 'NaN')
# def mode(slice_batch_size, batch_size, has_aug):
#     return f"slice_batch_size={slice_batch_size}, batch_size={batch_size}, has_aug={has_aug}"
# df.augment(has_aug)
# df.augment(mode)

g = sns.relplot(
    data=df,
    x='epoch',
    y='ece_loss',
    col='phase',
    hue='model',
    kind='line',
    height=12,
    )

# Set column spacing
g.fig.subplots_adjust(wspace=0.05)
# Set the y-axis limits
g.set(ylim=(0.0004, 0.0025))

In [ ]:
g = sns.relplot(
    data=df,
    x='epoch',
    y='elm_loss',
    col='phase',
    hue='model',
    kind='line',
    height=12,
    )

# Set column spacing
g.fig.subplots_adjust(wspace=0.05)
# Set the y-axis limits
g.set(ylim=(0.0004, 0.0025))

In [ ]:
g = sns.relplot(
    data=df,
    x='epoch',
    y='loss',
    col='phase',
    hue='model',
    kind='line',
    height=12,
    )

# Set column spacing
g.fig.subplots_adjust(wspace=0.05)
# Set the y-axis limits
g.set(ylim=(0.004, 0.012))